<a href="https://colab.research.google.com/github/gioalvaro/Coursera_Capstone/blob/master/Cluster_Canada_Neighborhood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Part 1: Retrieving Data

## Importing Libraries

In [0]:
import numpy as np # biblioteca para manejar datos vectorizados

import pandas as pd # biblioteca para analisis de datos
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import json # biblioteca para manejar archivos JSON

#!pip install geopy 
from geopy.geocoders import Nominatim # convierte direcciones en valores de longitud y latitud

import requests # bibliotecas para manejar request
from pandas.io.json import json_normalize # transforma JSON en tablas de pandas

# modulos de ploteo
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium 
import folium # biblioteca para renderizar mapas

from urllib.request import urlopen
from bs4 import BeautifulSoup

print('Biblioteca importadas.')

Biblioteca importadas.


## Retrieving from Wikipedia Postal Codes of Canada

In [0]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

html = urlopen(url)

In [0]:
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [0]:
text = soup.get_text()


In [0]:
rows = soup.find_all('tr')
print(rows[:10])

[<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>, <tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>, <tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>, <tr>
<td>M6A</td>
<td><a href="/wiki/North_York" ti

In [0]:
for row in rows:
  row_td = row.find_all('td')
  print(row_td)
  type(row_td)

[]
[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
</td>]
[<td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
</td>]
[<td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td>]
[<td>M4A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td>]
[<td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td>]
[<td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td>]
[<td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heig

In [0]:
import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

[A
, B
, C
, E
, G
, H
, J
, K
, L
, M
, N
, P
, R
, S
, T
, V
, X
, Y
]


str

In [0]:
list_rows = list_rows[1:len(list_rows)-5]

In [0]:
df = pd.DataFrame(list_rows)
df.head(10)

,0
0,"[M1A, Not assigned, Not assigned\n]"
1,"[M2A, Not assigned, Not assigned\n]"
2,"[M3A, North York, Parkwoods\n]"
3,"[M4A, North York, Victoria Village\n]"
4,"[M5A, Downtown Toronto, Harbourfront\n]"
5,"[M5A, Downtown Toronto, Regent Park\n]"
6,"[M6A, North York, Lawrence Heights\n]"
7,"[M6A, North York, Lawrence Manor\n]"
8,"[M7A, Queen's Park, Not assigned\n]"
9,"[M8A, Not assigned, Not assigned\n]"


## Preprocessing the retrieved data

>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood


In [0]:
df1 = df[0].str.split(',', expand=True)
df1.head(10)
df1.columns = ['PostalCode', 'Borough', 'Neighborhood']
df1.head(10)

,PostalCode,Borough,Neighborhood
0,[M1A,Not assigned,Not assigned\n]
1,[M2A,Not assigned,Not assigned\n]
2,[M3A,North York,Parkwoods\n]
3,[M4A,North York,Victoria Village\n]
4,[M5A,Downtown Toronto,Harbourfront\n]
5,[M5A,Downtown Toronto,Regent Park\n]
6,[M6A,North York,Lawrence Heights\n]
7,[M6A,North York,Lawrence Manor\n]
8,[M7A,Queen's Park,Not assigned\n]
9,[M8A,Not assigned,Not assigned\n]


In [0]:
df1.Borough = df1.Borough.astype('str')

>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [0]:
df_canada = df1[~df1.Borough.str.contains('Not assigned')].copy()

In [0]:
df_canada.PostalCode = df_canada.PostalCode.str.replace('[','')
df_canada.Neighborhood = df_canada.Neighborhood.str.replace('\n]','')

In [0]:
df_canada.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights



> More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.



In [0]:
df_canada_grouped = df_canada.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x))

> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.


In [0]:
df_canada_grouped = df_canada_grouped.reset_index()

>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [0]:
df_canada_NotAssigned = df_canada_grouped[df_canada_grouped.Neighborhood.str.contains('Not assigned')]

In [0]:
df_canada_NotAssigned.Neighborhood[:] = df_canada_NotAssigned.Borough[:]


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:1233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [0]:
df_canada_NotAssigned

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [0]:
replace_values = df_canada_NotAssigned.index.values

In [0]:
df_canada_grouped.iloc[replace_values] = df_canada_NotAssigned

>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


In [0]:
df_canada_grouped.shape

(103, 3)

## Part 2: Geocoder

### Importing Geocoder

In [0]:
!pip install geocoder

### Using Geocoder, but if It will not retrieve the coordinates, IBM has just provided in a csv file through an url 

In [0]:
import geocoder # import geocoder

lat = []
lon = []
for postal_code in df_canada_grouped.PostalCode:
  # initialize your variable to None
  lat_lng_coords = None
  # loop until you get the coordinates
  while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

  latitude = lat_lng_coords[0]
  longitude = lat_lng_coords[1]
  lat.append(latitude)
  lon.append(longitude)

KeyboardInterrupt: ignored

In [0]:
print(lat)
print(lon)

[]
[]


In [0]:
Geospatial_data = pd.read_csv('https://cocl.us/Geospatial_data')

In [0]:
Geospatial_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [0]:
df_geolocation = df_canada_grouped.merge(Geospatial_data, how = 'inner', left_on = 'PostalCode', right_on = 'Postal Code')

In [0]:
df_geolocation = df_geolocation.drop(['Postal Code'], axis = 1)

In [0]:
df_geolocation.shape

(103, 5)